In [1]:
# Gerekli kütüphaneleri import edelim
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor  # CatBoost'u ekledik

import warnings
warnings.filterwarnings('ignore')

# 1. Veri Setini Yükleme
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_x.csv')

# 2. Veri Ön İşleme ve Temizleme

# a. Yaş hesaplama ve 'Yas' sütununu ekleme
def calculate_age(born):
    try:
        born = pd.to_datetime(born, errors='coerce', infer_datetime_format=True)
        today = pd.to_datetime('today')
        age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))
        return int(age)
    except:
        return np.nan

# Yaşları hesaplayalım
train_df['Yas'] = train_df['Dogum Tarihi'].apply(calculate_age)
test_df['Yas'] = test_df['Dogum Tarihi'].apply(calculate_age)

# Üniversite okuma sayısını hesaplama
def calculate_university_count(row):
    if pd.isna(row['Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']):
        return 1  # Sadece şu anki üniversite var
    else:
        return 2  # Hem şu anki üniversite hem de önceki var

# Yeni sütunu ekleyelim
train_df['Universite Okuma Sayisi'] = train_df.apply(calculate_university_count, axis=1)
test_df['Universite Okuma Sayisi'] = test_df.apply(calculate_university_count, axis=1)

# b. Gereksiz sütunları çıkarma
columns_to_drop = ['Dogum Tarihi', 'Dogum Yeri', 'Lise Adi', 'Lise Adi Diger', 'Lise Sehir',
                   'Lise Bolum Diger', 'Lise Bolumu', 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

# c. Binary sütunların kodlanması
binary_columns = [
    'Burs Aliyor mu?', 'Daha Once Baska Bir Universiteden Mezun Olmus',
    'Baska Bir Kurumdan Burs Aliyor mu?', 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
    'Profesyonel Bir Spor Daliyla Mesgul musunuz?', 'Aktif olarak bir STK üyesi misiniz?',
    'Stk Projesine Katildiniz Mi?', 'Girisimcilikle Ilgili Deneyiminiz Var Mi?', 'Ingilizce Biliyor musunuz?'
]

for col in binary_columns:
    train_df[col] = train_df[col].map({'Evet': 1, 'Hayır': 0})
    test_df[col] = test_df[col].map({'Evet': 1, 'Hayır': 0})

# 'Cinsiyet' sütunu için kodlama
train_df['Cinsiyet'] = train_df['Cinsiyet'].map({'Erkek': 2, 'Kadın': 1, 'ERKEK': 2, 'KADIN': 1, 'Belirtmek istemiyorum': 0})
test_df['Cinsiyet'] = test_df['Cinsiyet'].map({'Erkek': 2, 'Kadın': 1, 'ERKEK': 2, 'KADIN': 1, 'Belirtmek istemiyorum': 0})

# 'Universite Turu' sütunu için kodlama
train_df['Universite Turu'] = train_df['Universite Turu'].map({'Özel': 1, 'Devlet': 0})
test_df['Universite Turu'] = test_df['Universite Turu'].map({'Özel': 1, 'Devlet': 0})

# d. Kategorik değişkenlerin kodlanması
categorical_columns = train_df.select_dtypes(include=['object']).columns

# Ortak sütunlar için birleşik veri seti oluşturma (Label Encoding için)
combined_df = pd.concat([train_df[categorical_columns], test_df[categorical_columns]], axis=0)

le = LabelEncoder()
for col in categorical_columns:
    combined_df[col] = combined_df[col].astype(str)
    combined_df[col] = le.fit_transform(combined_df[col])

# Tekrar ayırma
train_df[categorical_columns] = combined_df.iloc[:len(train_df)][categorical_columns]
test_df[categorical_columns] = combined_df.iloc[len(train_df):][categorical_columns]

# e. Eksik değerlerin doldurulması
# Eğitim verisindeki eksik değerleri dolduralım
for col in train_df.columns:
    if train_df[col].isnull().sum() > 0:
        if train_df[col].dtype in ['float64', 'int64']:
            median = train_df[col].median()
            train_df[col].fillna(median, inplace=True)
        else:
            mode = train_df[col].mode()[0]
            train_df[col].fillna(mode, inplace=True)

# Test verisindeki eksik değerleri eğitim verisiyle dolduralım
for col in test_df.columns:
    if test_df[col].isnull().sum() > 0:
        if test_df[col].dtype in ['float64', 'int64']:
            median = train_df[col].median()  # Eğitim setindeki medyanı kullan
            test_df[col].fillna(median, inplace=True)
        else:
            mode = train_df[col].mode()[0]  # Eğitim setindeki en sık değeri kullan
            test_df[col].fillna(mode, inplace=True)

# 3. Özellik Mühendisliği

# a. Yeni özellikler

# Ebeveynlerin Eğitim Ortalaması
train_df['Ebeveyn Egitim Ort'] = (train_df['Anne Egitim Durumu'] + train_df['Baba Egitim Durumu']) / 2
test_df['Ebeveyn Egitim Ort'] = (test_df['Anne Egitim Durumu'] + test_df['Baba Egitim Durumu']) / 2

# 1. Ebeveyn Çalışma Durumu
def map_working_status(status):
    if pd.isnull(status):
        return 0
    status = str(status).lower()
    if status in ['evet', 'çalışıyor', 'evet çalışıyor']:
        return 1
    else:
        return 0

for df in [train_df, test_df]:
    df['Anne Calisiyor'] = df['Anne Calisma Durumu'].apply(map_working_status)
    df['Baba Calisiyor'] = df['Baba Calisma Durumu'].apply(map_working_status)
    
    # Create 'Ebeveyn Calisma Durumu'
    def parents_working_status(row):
        if row['Anne Calisiyor'] == 1 and row['Baba Calisiyor'] == 1:
            return 2  # İkisi de çalışıyor
        elif row['Anne Calisiyor'] == 1 or row['Baba Calisiyor'] == 1:
            return 1  # Biri çalışıyor
        else:
            return 0  # Hiçbiri çalışmıyor
    
    df['Ebeveyn Calisma Durumu'] = df.apply(parents_working_status, axis=1)

# 2. Aile Geliri
education_mapping = {
    'Eğitim Yok': 0,
    'İlkokul Mezunu': 1,
    'Ortaokul Mezunu': 2,
    'Lise': 3,
    'Üniversite': 4,
    'Yüksek Lisans / Doktora': 5
}

sector_mapping = {
    'Kamu': 2,
    'Özel Sektör': 2,
    'Emekli': 1,
    'Ev Hanımı': 0,
    '0': 0,
    'Diğer': 1,
    np.nan: 0
}

for df in [train_df, test_df]:
    # Map education levels
    df['Anne Egitim Seviyesi'] = df['Anne Egitim Durumu'].map(education_mapping).fillna(0)
    df['Baba Egitim Seviyesi'] = df['Baba Egitim Durumu'].map(education_mapping).fillna(0)
    
    # Map sectors
    df['Anne Sektor Skoru'] = df['Anne Sektor'].map(sector_mapping).fillna(0)
    df['Baba Sektor Skoru'] = df['Baba Sektor'].map(sector_mapping).fillna(0)
    
    # 'Anne Calisiyor' ve 'Baba Calisiyor' zaten oluşturuldu
    
    # Calculate income score for each parent
    df['Anne Gelir Skoru'] = df['Anne Egitim Seviyesi'] * df['Anne Sektor Skoru'] * df['Anne Calisiyor']
    df['Baba Gelir Skoru'] = df['Baba Egitim Seviyesi'] * df['Baba Sektor Skoru'] * df['Baba Calisiyor']
    
    # Total family income score
    df['Aile Geliri'] = df['Anne Gelir Skoru'] + df['Baba Gelir Skoru']

# 3. Şehir ve Üniversite Puanı

# Üniversite Puanı
university_scores = train_df.groupby('Universite Adi')['Degerlendirme Puani'].mean().to_dict()

train_df['Universite Puani'] = train_df['Universite Adi'].map(university_scores)
test_df['Universite Puani'] = test_df['Universite Adi'].map(university_scores)

mean_univ_score = train_df['Universite Puani'].mean()
test_df['Universite Puani'].fillna(mean_univ_score, inplace=True)

# Şehir Puanı
city_scores = train_df.groupby('Ikametgah Sehri')['Degerlendirme Puani'].mean().to_dict()

train_df['Sehir Puani'] = train_df['Ikametgah Sehri'].map(city_scores)
test_df['Sehir Puani'] = test_df['Ikametgah Sehri'].map(city_scores)

mean_city_score = train_df['Sehir Puani'].mean()
test_df['Sehir Puani'].fillna(mean_city_score, inplace=True)

# 4. Lise Türü Puanı
lise_turu_scores = train_df.groupby('Lise Turu')['Degerlendirme Puani'].mean().to_dict()

train_df['Lise Turu Puani'] = train_df['Lise Turu'].map(lise_turu_scores)
test_df['Lise Turu Puani'] = test_df['Lise Turu'].map(lise_turu_scores)

mean_lise_turu_score = train_df['Lise Turu Puani'].mean()
test_df['Lise Turu Puani'].fillna(mean_lise_turu_score, inplace=True)

# 5. STK Deneyimi
for df in [train_df, test_df]:
    df['STK Deneyimi'] = df['Aktif olarak bir STK üyesi misiniz?'] + df['Stk Projesine Katildiniz Mi?']

# 4. Modelleme

# a. Hedef değişkeni ve özellikleri ayırma
X = train_df.drop(['Degerlendirme Puani', 'id'], axis=1)
y = train_df['Degerlendirme Puani']

# b. Eğitim ve test setlerinin sütunlarını eşleştirme
test_features = test_df.drop('id', axis=1)
common_cols = X.columns.intersection(test_features.columns)
X = X[common_cols]
test_features = test_features[common_cols]

# c. Cross-Validation ve Model Seçimi

# Kullanılacak modeller ve hiperparametreler
models = {
    'XGBoost': {
        'model': xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1]
        }
    },
    'CatBoost': {
        'model': CatBoostRegressor(verbose=0, random_state=42),  # CatBoost modelini ekledik
        'params': {
            'iterations': [100, 200],
            'depth': [4, 6, 8],
            'learning_rate': [0.01, 0.1],
            'l2_leaf_reg': [1, 3, 5]
        }
    }
}

# Performans sonuçlarını saklamak için
model_scores = []

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for model_name, mp in models.items():
    print(f"{model_name} modeli için eğitim yapılıyor...")
    grid_search = GridSearchCV(
        estimator=mp['model'],
        param_grid=mp['params'],
        cv=kf,
        scoring='neg_mean_squared_error',
        n_jobs=1,  # Paralel işleme kapatıldı
        verbose=1
    )
    grid_search.fit(X, y)
    best_rmse = np.sqrt(-grid_search.best_score_)
    print(f"{model_name} için en iyi RMSE: {best_rmse}")
    print(f"En iyi hiperparametreler: {grid_search.best_params_}\n")
    model_scores.append({
        'Model': model_name,
        'Best_RMSE': best_rmse,
        'Best_Params': grid_search.best_params_,
        'Best_Estimator': grid_search.best_estimator_
    })

# Sonuçları DataFrame olarak saklayalım
results_df = pd.DataFrame(model_scores).sort_values(by='Best_RMSE')
print("Model Karşılaştırma Sonuçları:")
print(results_df[['Model', 'Best_RMSE']])

# En iyi modeli seçelim
best_model_info = results_df.iloc[0]
best_model = best_model_info['Best_Estimator']
print(f"\nEn iyi model: {best_model_info['Model']}")

# 5. En İyi Model ile Tahmin ve Sonuç Dosyasının Oluşturulması

# Tüm veri üzerinde (X, y) en iyi modeli eğitelim
best_model.fit(X, y)

# Test verisi üzerinde tahmin yapalım
test_predictions = best_model.predict(test_features)

# Negatif tahminleri 0'a eşitleyelim (Eğer gerekiyorsa)
test_predictions = np.where(test_predictions < 0, 0, test_predictions)

submission = pd.DataFrame({
    'id': test_df['id'],
    'Degerlendirme Puani': test_predictions
})

submission.to_csv('submission.csv', index=False)
print("\nTahminler 'submission.csv' dosyasına kaydedildi.")


XGBoost modeli için eğitim yapılıyor...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
XGBoost için en iyi RMSE: 5.522961190798652
En iyi hiperparametreler: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}

CatBoost modeli için eğitim yapılıyor...
Fitting 5 folds for each of 36 candidates, totalling 180 fits
CatBoost için en iyi RMSE: 5.5053772651954445
En iyi hiperparametreler: {'depth': 8, 'iterations': 200, 'l2_leaf_reg': 3, 'learning_rate': 0.1}

Model Karşılaştırma Sonuçları:
      Model  Best_RMSE
1  CatBoost   5.505377
0   XGBoost   5.522961

En iyi model: CatBoost

Tahminler 'submission.csv' dosyasına kaydedildi.
